<a href="https://colab.research.google.com/github/AtogmatomaProgramming/OdonScanAI/blob/develop/manage_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *manage_frames.ipynb*

Breve descripción: script para la extracción de frames desde los videos que se grabaron para cada especie.

## 1. Peparar librerías y espacio de trabajo

In [ ]:
# Importación de librerías
from google.colab import drive
import os
import cv2

In [ ]:
# Establecer conexión con directorio de trabajo
drive.mount('/content/drive')

In [ ]:
# Rutas carpetas de trabajo
base_video_dir = "/content/drive/My Drive/alpha_training/videos"
output_frame_dir = "/content/drive/My Drive/alpha_training/frames"

## 2. Extracción de frames

In [ ]:
# Número de frames que quieres extraer por video
total_frames_desired = 802

In [ ]:
# Función para crear una carpeta en el directorio de salida (si no existe)
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
# Bucles de trabajo

## a) Recorrer las carpetas de especies en la carpeta de videos

for species in os.listdir(base_video_dir):
    species_path = os.path.join(base_video_dir, species)
    if os.path.isdir(species_path):
        # Crear una carpeta de salida para la especie en 'frames'
        # a través de la función "create_folder_if_not_exists"
        species_output_dir = os.path.join(output_frame_dir, species)
        create_folder_if_not_exists(species_output_dir)

        # Recorrer los videos en la carpeta de cada especie
        for idx, video_file in enumerate(os.listdir(species_path)):
            video_path = os.path.join(species_path, video_file)
            video_code = f'v{idx + 1}'

            # Verificar si ya existen frames del video en la carpeta de la especie
            existing_frames = [f for f in os.listdir(species_output_dir) if f.startswith(video_code)]
            if existing_frames:
                print(f"Frames ya extraídos para {video_file}, ignorando...")
                continue

            # Procesar el video y calcular el intervalo para los frames deseados
            cap = cv2.VideoCapture(video_path)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            interval = max(1, frame_count // total_frames_desired)

            # Extraer frames en intervalos calculados
            current_frame = 0
            saved_frames = 0
            while cap.isOpened() and saved_frames < total_frames_desired:
                ret, frame = cap.read()
                if not ret:
                    break
                if current_frame % interval == 0:
                    frame_name = os.path.join(species_output_dir, f'{video_code}_frame{current_frame}.jpg')
                    cv2.imwrite(frame_name, frame)
                    saved_frames += 1
                current_frame += 1

            cap.release()
            # Visualización por pantalla de los frames extraídos por video
            print(f"{saved_frames} frames extraídos de {video_file} y guardados en {species_output_dir}")

## b) Confrimación de la extracción final de frames

print("Extracción de frames completada.")